<a href="https://colab.research.google.com/github/sangjinsu/personalized-recommendation-system/blob/main/MF_%EA%B8%B0%EB%B0%98_%EC%B6%94%EC%B2%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 메모리 기반 알고리즘

- 메모리에 있는 데이터를 계산해서 추천하는 방식
- 개별 사용자 데이터 집중
- 원래 데이터에 충실하게 사용
- 대규모 데이터에 느리게 반응

## 모델 기반 알고리즘

- 데이터로부터 미리 모델을 구성 후 필요 시 추천하는 방식
- 전체 사용자 패턴 집중
- 대규모 데이터에 빠르게 반응
- 모델 생성 과정 오래 걸림 

## Matrix Factorization 방식의 원리 

### SGD Stochastic Gradient Decent 를 사용한  MF 알고리즘

1. 잠재 요인  K 선택
2. P, Q 행렬 초기화
3. 예측 평점 R_hat 계산
4. 실제 R 과 R_hat 간 오차 계산 및 P, Q 수정
5. 기존 오차 도달 확인 


### SGD 를 사용한 MF 기본 알고리즘

In [1]:
import os
import pandas as pd
import numpy as np

base_src = ''
u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(u_data_src, sep='\t',
                      names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)

ratings

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [2]:
class MF():
  def __init__(self, ratings, hyper_params):
    self.R = np.array(ratings)
    self.num_users, self.num_items = np.shape(self.R)
    self.K = hyper_params['K']
    self.alpha = hyper_params['alpha']
    self.beta = hyper_params['beta']
    self.iterations = hyper_params['iterations']
    self.verbose = hyper_params['verbose']

  def rmse(self):
    xs, ys = self.R.nonzero()
    self.predictions = []
    self.errors = [] 

    for x, y in zip(xs, ys):
      prediction = self.get_prediction(x, y)
      self.predictions.append(prediction)
      self.errors.append(self.R[x,y] - prediction)
    self.prediction = np.array(self.predictions)
    self.errors = np.array(self.errors)

    return np.sqrt(np.mean(self.errors ** 2))

  def train(self):
    self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
    self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

    self.b_u = np.zeros(self.num_users)
    self.b_d = np.zeros(self.num_items)
    self.b = np.mean(self.R[self.R.nonzero()])

    rows, columns = self.R.nonzero()
    self.samples = [(i,j,self.R[i,j]) for i, j in zip(rows, columns)]

    training_process = []
    for i in range(self.iterations):
      np.random.shuffle(self.samples)
      self.sgd()
      rmse = self.rmse()
      training_process.append((i+1, rmse))
      if self.verbose:
        if (i+1) % 10 == 0:
          print('Iteration : %d ; train RMSE = %.4f' % (i+1, rmse))
    return training_process

  def get_prediction(self, i, j):
    prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i,:].dot(self.Q[j,].T)
    return prediction

  def sgd(self):
    for i, j, r in self.samples:
      prediction = self.get_prediction(i, j)
      e = (r-prediction)
      self.b_u[i] += self.alpha * (e - (self.beta * self.b_u[i]))
      self.b_d[j] += self.alpha * (e - (self.beta * self.b_d[j]))

      self.P[i, :] += self.alpha * ((e * self.Q[j,:]) - (self.beta * self.P[i, :]))
      self.Q[j, :] += self.alpha * ((e * self.P[i,:]) - (self.beta * self.Q[j, :]))

R_temp = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

hyper_params = {
    'K' : 30,
    'alpha' : 0.001,
    'beta' : 0.02,
    'iterations' : 100,
    'verbose': True
}


mf = MF(R_temp, hyper_params)

train_process = mf.train()

Iteration : 10 ; train RMSE = 0.9585
Iteration : 20 ; train RMSE = 0.9373
Iteration : 30 ; train RMSE = 0.9280
Iteration : 40 ; train RMSE = 0.9224
Iteration : 50 ; train RMSE = 0.9181
Iteration : 60 ; train RMSE = 0.9138
Iteration : 70 ; train RMSE = 0.9085
Iteration : 80 ; train RMSE = 0.9013
Iteration : 90 ; train RMSE = 0.8913
Iteration : 100 ; train RMSE = 0.8792


In [3]:
import os
import pandas as pd
import numpy as np

base_src = ''
u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(u_data_src, sep='\t',
                      names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)

# train / test set 분리
from sklearn.utils import shuffle 
TRAIN_SIZE = 0.75
# (사용자 - 영화 - 평점)
ratings = shuffle(ratings, random_state= 2021)
cutoff = int(TRAIN_SIZE * len(ratings))
ratings_train = ratings.iloc[:cutoff]
ratings_test = ratings.iloc[cutoff:]

class NEW_MF():
  def __init__(self, ratings, hyper_params):
    self.R = np.array(ratings)
    # 사용자 수 (num_users) 와 아이템 수(num_items)를 받아온다
    self.num_users, self.num_items = np.shape(self.R)
    # 아래는 MF weight 조절을 위한 하이퍼파라미터이다
    # K : 잠재요인(latent factor)의 수
    self.K = hyper_params['K']
    # alpha 학습률
    self.alpha = hyper_params['alpha']
    # beta  정규화 계수
    self.beta = hyper_params['beta']
    # iterations SGD 계산을 할 때의 반복 횟수
    self.iterations = hyper_params['iterations']
    # verbose SGD 학습 과정을 중간중간에 출력할 것인지에 대한 여부
    self.verbose = hyper_params['verbose']

    item_id_index = []
    index_item_id = []
    for i, one_id in enumerate(ratings):
      item_id_index.append([one_id, i])
      index_item_id.append([i, one_id])
    self.item_id_index = dict(item_id_index)
    self.index_item_id = dict(index_item_id)

    user_id_index = []
    index_user_id = []
    for i, one_id in enumerate(ratings.T):
      user_id_index.append([one_id, i])
      index_user_id.append([i, one_id])
    self.user_id_index = dict(user_id_index)
    self.index_user_id = dict(index_user_id)    


  def rmse(self):
    # self.R 에서 평점이 있는 요소의 인덱스를 가져온다 
    xs, ys = self.R.nonzero()
    # prediction 과 error를 담을 리스트 변수 초기화 
    self.predictions = []
    self.errors = [] 
    # 평점이 있는 요소 (사용자 x, 아이템 y) 각각에 대해서 아래의 코드를 실행한다
    for x, y in zip(xs, ys):
      # 사용자 x, 아이템 y 에 대해 평점 예측치를 get_prediction() 함수를 사용해서 계산한다.
      prediction = self.get_prediction(x, y)
      # 예측값을 예측값 리스트에 추가한다 
      self.predictions.append(prediction)
      # 실제값 과 예측값의 차이를 계산해서 오차값 리스트에 추가한다
      self.errors.append(self.R[x,y] - prediction)
    # 예측값 리스트와 오차값 리스트를 numpy array 형태로 변환한다.
    self.prediction = np.array(self.predictions)
    self.errors = np.array(self.errors)
    # error를 활용해서 RMSE 도출 
    return np.sqrt(np.mean(self.errors ** 2))

  def sgd(self):
    for i, j, r in self.samples:
      # 사용자 i, 아이템 j 에 대한 평저 예측치 계산
      prediction = self.get_prediction(i, j)
      # 실제 평점과 비교한 오차 계산
      e = (r-prediction)

      # 사용자 평가 경향 계산 및 업데이트
      self.b_u[i] += self.alpha * (e - (self.beta * self.b_u[i]))
      # 아이템 평가 경향 계산 및 업데이트
      self.b_d[j] += self.alpha * (e - (self.beta * self.b_d[j]))

      # P 행렬 계산 및 업데이트
      self.P[i, :] += self.alpha * ((e * self.Q[j,:]) - (self.beta * self.P[i, :]))
      # Q 행렬 계산 및 업데이트
      self.Q[j, :] += self.alpha * ((e * self.P[i,:]) - (self.beta * self.Q[j, :]))
  
  def get_prediction(self, i, j):
    # 사용자 i, 아이템 j에 대한 평점 예측치를 앞에서 배웠던 식을 이용해서 구한다 
    prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i,:].dot(self.Q[j,].T)
    return prediction
 
  #  Test set 선정
  def set_test(self, ratings_test):
    test_set = []
    for i in range(len(ratings_test)):
      x = self.user_id_index[ratings_test.iloc[i, 0]]
      y = self.item_id_index[ratings_test.iloc[i, 1]]
      z = ratings_test.iloc[i, 2]
      test_set.append([x, y, z])
      self.R[x, y] = 0
    self.test_set = test_set 
    return test_set

  # Test set RMSE 계산 
  def test_rmse(self):
    error = 0
    for one_set in self.test_set:
      predicted = self.get_prediction(one_set[0], one_set[1])
      # e => e^2
      error += pow(one_set[2] - predicted, 2)
    return np.sqrt(error/len(self.test_set))

  def test(self):
    self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
    self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

    self.b_u = np.zeros(self.num_users)
    self.b_d = np.zeros(self.num_items)
    self.b = np.mean(self.R[self.R.nonzero()])

    rows, columns = self.R.nonzero()
    self.samples = [(i,j,self.R[i,j]) for i, j in zip(rows, columns)]

    training_process = []
    for i in range(self.iterations):
      np.random.shuffle(self.samples)
      self.sgd()
      rmse1 = self.rmse()
      rmse2 = self.test_rmse()
      training_process.append((i+1, rmse1, rmse2))
      if self.verbose:
        if (i+1) % 10 == 0:
          print('Iteration : %d ; train RMSE = %.4f ; TEST RMSE = %.4f' % (i+1, rmse1, rmse2))
    return training_process

  def get_one_prediction(self, user_id, item_id):
    return self.get_prediction(self.user_id_index[user_id],
                               self.item_id_index[item_id])
    
  def full_prediction(self):
    return self.b + self.b_u[:,np.newaxis] + self.b_d[np.newaxis,:] + self.P.dot(self.Q.T)


R_temp = ratings.pivot(index='user_id',
                       columns='movie_id',
                       values='rating').fillna(0)

hyper_params = {
    'K' : 30,
    'alpha' : 0.001,
    'beta' : 0.02,
    'iterations' : 100,
    'verbose': True
}


mf = NEW_MF(R_temp, hyper_params)
train_set = mf.set_test(ratings_test)
result = mf.test()
result

Iteration : 10 ; train RMSE = 0.9666 ; TEST RMSE = 0.9807
Iteration : 20 ; train RMSE = 0.9413 ; TEST RMSE = 0.9623
Iteration : 30 ; train RMSE = 0.9298 ; TEST RMSE = 0.9552
Iteration : 40 ; train RMSE = 0.9230 ; TEST RMSE = 0.9515
Iteration : 50 ; train RMSE = 0.9182 ; TEST RMSE = 0.9494
Iteration : 60 ; train RMSE = 0.9142 ; TEST RMSE = 0.9479
Iteration : 70 ; train RMSE = 0.9106 ; TEST RMSE = 0.9468
Iteration : 80 ; train RMSE = 0.9067 ; TEST RMSE = 0.9459
Iteration : 90 ; train RMSE = 0.9021 ; TEST RMSE = 0.9448
Iteration : 100 ; train RMSE = 0.8961 ; TEST RMSE = 0.9435


[(1, 1.0785224669415268, 1.0810883180253734),
 (2, 1.0480755764231775, 1.0523049339833404),
 (3, 1.0269799989424806, 1.0327824094188098),
 (4, 1.0115176391292917, 1.018793310416272),
 (5, 0.9996899402083465, 1.0083087507168345),
 (6, 0.9903576170450166, 1.0002334908098556),
 (7, 0.982783777890486, 0.9938203542639089),
 (8, 0.9765005861264024, 0.988626025417489),
 (9, 0.9711887240752084, 0.9843358484028454),
 (10, 0.9666443468775852, 0.9807378201019957),
 (11, 0.9626891236191615, 0.977648216069924),
 (12, 0.959210753724076, 0.974990464941052),
 (13, 0.9561209090199669, 0.9726841173518862),
 (14, 0.9533523261330438, 0.970674925257913),
 (15, 0.9508567698494444, 0.9688626198731679),
 (16, 0.9485947984536609, 0.9672591090424103),
 (17, 0.9465278594848886, 0.9658220292443617),
 (18, 0.944632407897325, 0.9645274764142795),
 (19, 0.9428851671171764, 0.9633605590085933),
 (20, 0.9412666070260144, 0.9622725448426546),
 (21, 0.9397674115472686, 0.961280433142841),
 (22, 0.9383684663346652, 0.960

In [4]:
print(mf.full_prediction())

[[3.86537168 3.33510059 2.98070658 ... 3.3403474  3.51614261 3.45028494]
 [3.81973131 3.26392103 2.86892754 ... 3.2360854  3.39346054 3.34003093]
 [3.38594637 2.88840291 2.48364429 ... 2.86793533 2.96284583 2.96280981]
 ...
 [4.15317503 3.61155059 3.21435547 ... 3.5918239  3.70952484 3.68212601]
 [4.2658625  3.77412788 3.42401544 ... 3.73737618 3.85586591 3.85802128]
 [3.81978059 3.33747599 2.90426211 ... 3.29777249 3.42116205 3.37919269]]


In [5]:
print(mf.get_one_prediction(1, 2))

3.3351005870385615


### MF 최적의 파라미터 찾기  

1. 대략적인 최적의 K 위치 찾기
2. 대략적 K 주변 탐색으로 최적 K 찾기
3. 주어진 K 통해 최적의 iterations 선택 

In [9]:
# 최적의 K 값 찾기 
results = []
index = []

R_temp = ratings.pivot(index='user_id',
                       columns='movie_id',
                       values='rating').fillna(0)

#for K in range(50, 261, 10):
for K in range(50, 101, 10):
  print('K : ' + str(K))
  hyper_params = {
    'K' : K,
    'alpha' : 0.001,
    'beta' : 0.02,
    'iterations' : 100,
    'verbose': True
  }

  mf = NEW_MF(R_temp,
              hyper_params)
  test_set = mf.set_test(ratings_test)
  result = mf.test()
  index.append(K)
  results.append(result)

K : 50
Iteration : 10 ; train RMSE = 0.9669 ; TEST RMSE = 0.9807
Iteration : 20 ; train RMSE = 0.9417 ; TEST RMSE = 0.9622
Iteration : 30 ; train RMSE = 0.9305 ; TEST RMSE = 0.9552
Iteration : 40 ; train RMSE = 0.9239 ; TEST RMSE = 0.9515
Iteration : 50 ; train RMSE = 0.9194 ; TEST RMSE = 0.9493
Iteration : 60 ; train RMSE = 0.9160 ; TEST RMSE = 0.9479
Iteration : 70 ; train RMSE = 0.9129 ; TEST RMSE = 0.9468
Iteration : 80 ; train RMSE = 0.9098 ; TEST RMSE = 0.9459
Iteration : 90 ; train RMSE = 0.9061 ; TEST RMSE = 0.9448
Iteration : 100 ; train RMSE = 0.9014 ; TEST RMSE = 0.9435
K : 60
Iteration : 10 ; train RMSE = 0.9669 ; TEST RMSE = 0.9807
Iteration : 20 ; train RMSE = 0.9418 ; TEST RMSE = 0.9622
Iteration : 30 ; train RMSE = 0.9307 ; TEST RMSE = 0.9552
Iteration : 40 ; train RMSE = 0.9242 ; TEST RMSE = 0.9515
Iteration : 50 ; train RMSE = 0.9199 ; TEST RMSE = 0.9493
Iteration : 60 ; train RMSE = 0.9166 ; TEST RMSE = 0.9479
Iteration : 70 ; train RMSE = 0.9137 ; TEST RMSE = 0.9469

In [11]:
summary = []
for i in range(len(results)):
    RMSE = []
    for result in results[i]:
        RMSE.append(result[2])
    min = np.min(RMSE)
    j = RMSE.index(min)
    summary.append([index[i], j+1, RMSE[j]])

In [12]:
summary

[[50, 100, 0.9435043212456646],
 [60, 100, 0.9441512772312559],
 [70, 100, 0.9442276906124157],
 [80, 100, 0.94423395217587],
 [90, 100, 0.9440770520001189],
 [100, 100, 0.9444150606137733]]

### MF와 SVD

